In [148]:
from typing import List, Sequence
import math
import numpy as np

- Problema 1: Penalty 

<img src="figs/penaulty1.jpg" width="600"  alt="Superfície mínima linear">

In [149]:
def Penalti(x, a, b):
    # Somatório de (xi - 1)^2 para i = 1...n
    term1 = a * sum((xi - 1)**2 for xi in x)

    # Somatório de xi^2
    sum_x2 = sum(xi**2 for xi in x)

    # Aplicando toda a expressão
    term2 = b * (sum_x2 - 0.25)**2

    return term1 + term2

# Utilizar a = 1 e b = 10^-3
#link do paper, página 29 do paper e 32 do pdf: https://apps.dtic.mil/sti/tr/pdf/ADA078713.pdf

- Os problemas abaixo esão disponíveis no paper, a partir da página 11 do pdf (27 do paper)

https://dl.acm.org/doi/pdf/10.1145/355934.355936

- Problema 2: Trigonometric

<img src="figs/trigonometric.jpg" width="600"  alt="Trignomometric">

In [150]:
def trigonometric(x):
    """
    Função trigonométrica vetorial.
    x: lista ou vetor de tamanho n
    retorna: lista com f(x)
    """
    n = len(x)
    soma = sum(math.cos(xj) for xj in x)
    f = []
    for i in range(n):
        fi = n - soma + (i+1)*(1 - math.cos(x[i])) - math.sin(x[i])
        f.append(fi)
    return np.array(f)

# x0 = [1/n, ..., 1/n]
# Mínimo = f=0

# Definindo a funçao objetivo (o L-BFGS-B exige que seja uma funçao escalar, e a trigonometrica retorna um vetor)
def trigonometric_ext_objective(x):
  fx = trigonometric(x)
  return np.dot(fx, fx) # ||f(x)||²

Se f(x) = 0 (o ponto que você quer), então objective(x) = 0.

Se f(x) não for zero, objective(x) será positivo e vai indicar “quão longe” você está da solução.

Assim, o minimize pode buscar o x que faz objective(x) ser mínimo → ou seja, que faz f(x) ficar o mais próximo de zero possível.

- Problema 3: Extended Rosenbrock

<img src="figs/extendeRosembrock.png" width="600"  alt="ExtendentRosembrock">

In [151]:
def rosenbrock_ext_residuals(x: List[float]) -> List[float]:
    n = len(x)
    assert n % 2 == 0, "n deve ser par" # Para armezenar os residuos da funçao em pares, é necessário que seja par
    m = n // 2
    r = [0.0] * n
    for k in range(m):
        i = 2*k      # índice 0-based de x_{2k-1}
        r[2*k]   = 10.0 * (x[i+1] - x[i]**2)   # r_{2k-1}
        r[2*k+1] = 1.0 - x[i]                  # r_{2k}
    return np.array(r)

def rosenbrock_ext_objective(x: List[float]) -> float:
    r = rosenbrock_ext_residuals(x)
    return np.dot(r,r) # ||r||²

# Minimo = f=0 na origem

- Problema 4: Extended Powell

<img src="figs/extendedPowell.jpg" width="600"  alt="Extended Powell">

In [152]:
def powell_singular_residuals(x: List[float]) -> List[float]:
    """
    Calcula os resíduos da Extended Powell Singular Function.
    x: lista de tamanho n (deve ser múltiplo de 4)
    retorna: lista de resíduos r_j
    """
    n = len(x)
    assert n % 4 == 0, "O tamanho de x deve ser múltiplo de 4"
    m = n // 4
    r = [0.0] * n

    for i in range(m):
        # índices do bloco (0-based)
        i1 = 4*i
        i2 = i1 + 1
        i3 = i1 + 2
        i4 = i1 + 3

        r[i1] = x[i1] + 10 * x[i2]                 # f_{4i-3}
        r[i2] = (5**0.5) * (x[i3] - x[i4])         # f_{4i-2}
        r[i3] = (x[i2] - 2*x[i3])**2               # f_{4i-1}
        r[i4] = (10**0.5) * (x[i1] - x[i4])**2     # f_{4i}

    return np.array(r)

def powell_singular_ext_objective(x: List[float]) -> float:
    """
    Função objetivo: soma dos quadrados dos resíduos.
    """
    r = powell_singular_residuals(x)
    return np.dot(r,r) # ||r||²

# minimo = f=0 na origem

- As tabelas abaixo e as funções estão disponíveis no artigo abaixo, a partir da página 4 do pdf (843 do paper):
https://www.ams.org/journals/mcom/1978-32-143/S0025-5718-1978-0483452-7/S0025-5718-1978-0483452-7.pdf

In [153]:
a = [
    1.25, 1.40, 2.40, 1.40, 1.75, 1.20, 2.25, 1.20, 1.00, 1.10,
    1.50, 1.60, 1.25, 1.25, 1.20, 1.20, 1.40, 0.50, 0.50, 1.25,
    1.80, 0.75, 1.25, 1.40, 1.60, 2.00, 1.00, 1.60, 1.25, 2.75,
    1.25, 1.25, 1.25, 3.00, 1.50, 2.00, 1.25, 1.40, 1.80, 1.50,
    2.20, 1.40, 1.50, 1.25, 2.00, 1.50, 1.25, 1.40, 0.60, 1.50
]

B = [
    1.0, 1.5, 1.0, 0.1, 1.5, 2.0, 1.0, 1.5, 3.0, 2.0,
    1.0, 3.0, 0.1, 1.5, 0.15, 2.0, 1.0, 0.1, 3.0, 0.1,
    1.2, 1.0, 0.1, 2.0, 1.2, 3.0, 1.5, 3.0, 2.0, 1.0,
    1.2, 2.0, 1.0
]

d = [
    5.0, 5.0, 5.0, 2.5, 6.0, 6.0, 5.0, 6.0, 10.0, 6.0,
    5.0, 9.0, 2.0, 7.0, 2.5, 6.0, 5.0, 2.0, 9.0, 2.0,
    5.0, 5.0, 2.5, 5.0, 6.0, 10.0, 7.0, 10.0, 6.0, 5.0,
    4.0, 4.0, 4.0
]

A_sets = [
    [31], [1], [2], [4], [6], [8], [10], [12], [11,13,14], [16],
    [9,18], [5,20,21], [19], [23], [7,25], [28], [29], [32], [3,33], [35],
    [36], [30,37], [38,39], [40], [41], [44], [46], [42,45,48,50], [26,34,43], [15,17,24,47],
    [49], [22], [27]
]

B_sets = [
    [1], [2,3], [4,5], [6,7], [8,9], [10,11], [12,13], [14,15], [16,17], [18,19],
     [20], [], [22, 23, 24], [25,26], [27,28], [29,30], [31,32], [33,34], [35], [21,36],
    [37,38], [39], [40], [41,42], [43,44,50], [45,46,47], [48], [49], [], [],
    [], [], []

]

# Convertendo os conjuntos para indices 0-based. Dessa forma, o numero [50] agora é [49] (estava tendo problemas de indices)
A_sets = [[j-1 for j in s] for s in A_sets]
B_sets = [[j-1 for j in s] for s in B_sets]

- Problema 6: QOR

<img src="figs/QOR.jpg" width="600"  alt="QOR">

In [154]:
from typing import List, Sequence

def f_qor(
    x: Sequence[float],
    a: Sequence[float],
    B: Sequence[float],
    d: Sequence[float],
    A_sets: List[List[int]],
    B_sets: List[List[int]],
) -> float:
    """
    x: vetor de variáveis (índices 0-based)
    a: coeficientes para i=0..49  (50 itens)
    B, d: coeficientes para i=0..32 (33 itens)
    A_sets[i], B_sets[i]: listas de índices j (0-based) usados no i-ésimo termo da penalidade
    """
    # 1) sum_{i=1..50} a_i * x_i^2   -> i = 0..49 em Python
    term1 = sum(a[i] * (x[i] ** 2) for i in range(50))

    # 2) sum_{i=1..33} B_i * [ d_i - sum_{j in A(i)} x_j + sum_{j in B(i)} x_j ]^2
    term2 = 0.0
    for i in range(33):
        inner = d[i] \
              - sum(x[j] for j in A_sets[i]) \
              + sum(x[j] for j in B_sets[i])
        term2 += B[i] * (inner ** 2)

    return term1 + term2


# O ponto inicial é o vetor de zeros


- Problema 7: GOR

<img src="figs/GOR.jpg" width="600"  alt="GOR">

In [155]:
import math

def c_i(x_i, a_i):
    if x_i >= 0:
        return a_i * x_i * math.log(1 + x_i)
    else:
        return -a_i * x_i * math.log(1 - x_i)

def b_i(y_i, B_i):
    if y_i >= 0:
        return B_i * y_i**2 * math.log(1 + y_i)
    else:
        return B_i * y_i**2

def f_gor(x, a, B, d, A_sets, B_sets):
    term1 = sum(c_i(x[i], a[i]) for i in range(50))
    term2 = 0.0
    for i in range(33):
        y_i = d[i] - sum(x[j-1] for j in A_sets[i]) + sum(x[j-1] for j in B_sets[i])
        term2 += b_i(y_i, B[i])
    return term1 + term2

# O ponto inicial é o vetor de zeros

- Problema 8: PSP

<img src="figs/PSP.jpg" width="600"  alt="PSP">

In [156]:
def h_i(y_i):
    if y_i >= 0.1:
        return 1.0 / y_i
    else:
        return 100.0 * (0.1 - y_i) + 10.0

def f_psp(x, a, B, d, A_sets, B_sets):
    # Termo 1: soma dos a_i * (x_i - 5)^2 para i = 0..49 ( Python 0-based )
    term1 = sum(a[i] * (x[i] - 5.0) ** 2 for i in range(50))

    # Termo 2: soma dos B_i * h_i(y_i) para i = 0..32
    term2 = 0.0
    for i in range(33):
        y_i = d[i] \
              - sum(x[j - 1] for j in A_sets[i]) \
              + sum(x[j - 1] for j in B_sets[i])
        term2 += B[i] * h_i(y_i)

    return term1 + term2

# O ponto inicial é o vetor de zeros

- Problema 9: Tridiagonal

<img src="figs/tridiagonal.jpg" width="600"  alt="Tridiagonal">

In [157]:
def tridia_objective(x):

    n = len(x)
    
    # Primeiro termo: (x1 - 1)^2
    f = (x[0] - 1)**2
    
    # Termos subsequentes: (2*xi - x_{i-1})^2 para i = 2...n
    for i in range(1, n):
        f += (2*x[i] - x[i-1])**2
    
    return f

- Problema 10: Linear Minimum Surface

<img src="figs/lmsurface.jpg" width="600"  alt="Superfície mínima linear">

In [158]:
def lminsurf_objective(x):
    #x: vetor de variáveis (deve ser um quadrado perfeito, n = p^2)
        
    n = len(x)
    p = int(math.sqrt(n))
    
    if p*p != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 9:
        raise ValueError(f"n = {n} deve ser pelo menos 9")
    
    # Número de elementos
    nel = (p-1)**2
    
    f = 0
     # Para cada elemento (i,j) 
    for i in range(p-1):
        for j in range(p-1):
            # Índices dos 4 vértices do elemento
            idx1 = i*p + j           # (i,j)
            idx2 = i*p + (j+1)       # (i,j+1)
            idx3 = (i+1)*p + j       # (i+1,j)
            idx4 = (i+1)*p + (j+1)   # (i+1,j+1)
            
            # Cálculo da área do elemento
            a = x[idx1] - x[idx4]  # x(i,j) - x(i+1,j+1)
            b = x[idx3] - x[idx2]  # x(i+1,j) - x(i,j+1)
            
            ri = 1 + 0.5 * nel * (a**2 + b**2)
            f += math.sqrt(ri) / nel
    
    return f


def lminsurf_setup(n):

    p = int(math.sqrt(n))
    
    if p*p != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 9:
        raise ValueError(f"n = {n} deve ser pelo menos 9")
    
    h = 1.0 / (p-1)
    x0 = np.zeros(n)
    xlower = -np.inf * np.ones(n)
    xupper = np.inf * np.ones(n)
    
    # Configurar condições de contorno
    for iy in range(p):
        if iy == 0:  # Borda inferior
            for ix in range(p):
                t = ix * h
                x0[ix] = 1 + 8*t
                xlower[ix] = x0[ix]
                xupper[ix] = x0[ix]
        elif iy == p-1:  # Borda superior
            for ix in range(p):
                idx = ix + (p-1)*p
                t = ix * h
                x0[idx] = 5 + 8*t
                xlower[idx] = x0[idx]
                xupper[idx] = x0[idx]
        else:  # Bordas laterais
            # Borda esquerda
            idx = iy*p
            t = iy * h
            x0[idx] = 1 + 4*t
            xlower[idx] = x0[idx]
            xupper[idx] = x0[idx]
            
            # Borda direita
            idx = (iy+1)*p - 1
            x0[idx] = 9 + 4*t
            xlower[idx] = x0[idx]
            xupper[idx] = x0[idx]
    
    return x0, xlower, xupper



- Problema 11: Extended ENGVL1

<img src="figs/engval1.jpg" width="600"  alt="ENgVAL1">


In [159]:
def engval1_objective(x):

    n = len(x)
    f = 0
    
    # Para cada par consecutivo (xi, xi+1)
    for i in range(n-1):
        t = x[i]**2 + x[i+1]**2
        f += t**2 - 4*x[i] + 3
    
    return f
    

- Problema 12: Matrix Square Root 1


In [160]:

def msqrtals_objective(x):

    n = len(x)
    m = int(math.sqrt(n))
    
    if m*m != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Construir matriz B
    b = np.sin(np.arange(1, n+1)**2)
    B = b.reshape(m, m).T
    
    # Construir matriz A = B*B
    A = B @ B
    
    # Reshape x para matriz X
    X = x.reshape(m, m)
    
    # Calcular resíduos: A - X*X
    residual = A - X @ X
    
    # Soma dos quadrados dos resíduos
    f = np.sum(residual**2)
    
    return f

def msqrtals_setup(n=16):
    #n: número de variáveis (deve ser um quadrado perfeito, padrão 16)
    
    m = int(math.sqrt(n))
    
    if m*m != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Ponto inicial: 0.2*sin((1:n).^2)'
    x0 = 0.2 * np.sin(np.arange(1, n+1)**2)
    
    return x0

- Problema 13: Matrix Square Root 2

In [161]:

def msqrtbls_objective(x):
    n = len(x)
    m = int(math.sqrt(n))
    
    if m*m != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Construir matriz B (Case 1: b(2*m+1) = 0)
    b = np.sin(np.arange(1, n+1)**2)
    b[2*m] = 0  # Defines Case 1
    B = b.reshape(m, m).T
    
    # Construir matriz A = B*B
    A = B @ B
    
    # Reshape x para matriz X
    X = x.reshape(m, m)
    
    # Calcular resíduos: A - X*X
    residual = A - X @ X
    
    # Soma dos quadrados dos resíduos
    f = np.sum(residual**2)
    
    return f

def msqrtbls_setup(n=16):

    m = int(math.sqrt(n))
    
    if m*m != n:
        raise ValueError(f"n = {n} deve ser um quadrado perfeito")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Ponto inicial: 0.2*sin((1:n).^2)'
    x0 = 0.2 * np.sin(np.arange(1, n+1)**2)
    
    return x0


- Problema 14: Extended Freudentstein and Roth

<img src="figs/freuroth.jpg" width="600"  alt="Extended Freudentstein and Roth">

In [162]:
def freuroth_objective(x):

    n = len(x)
    f = 0
    
    # Para cada par consecutivo (xi, xi+1)
    for i in range(n-1):
        r1 = x[i] - 13 + 5*x[i+1]**2 - x[i+1]**3 - 2*x[i+1]
        r2 = x[i] - 29 + x[i+1]**3 + x[i+1]**2 - 14*x[i+1]
        f += r1**2 + r2**2
    
    return f


- Problema 15: Sparse Matrix Square Root

In [163]:
def spmsqrt_objective(x):

    n = len(x)
    m = (n + 2) // 3
    
    if 3*m - 2 != n:
        raise ValueError(f"n = {n} deve satisfazer n = 3m-2")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Construir matriz B tridiagonal
    B = np.zeros((m, m))
    b = np.sin(np.arange(1, 3*m-1)**2)
    
    ib = 0
    for j in range(m):
        if j == 0:
            B[0:2, 0] = b[0:2]
            ib = 2
        elif j == m-1:
            B[m-2:m, m-1] = b[ib:ib+2]
        else:
            B[j-1:j+2, j] = b[ib:ib+3]
            ib += 3
    
    # Construir matriz A = B*B
    A = B @ B
    
    # Extrair elementos não-zero de A (estrutura tridiagonal)
    a = []
    for j in range(m):
        start = max(0, j-2)
        end = min(m, j+3)
        a.extend(A[start:end, j])
    
    # Construir matriz X tridiagonal a partir de x
    X = np.zeros((m, m))
    ib = 0
    for j in range(m):
        if j == 0:
            X[0:2, 0] = x[0:2]
            ib = 2
        elif j == m-1:
            X[m-2:m, m-1] = x[n-2:n]
        else:
            X[j-1:j+2, j] = x[ib:ib+3]
            ib += 3
    
    # Calcular resíduos: A - X*X
    residual = A - X @ X
    
    # Extrair elementos não-zero do resíduo (estrutura tridiagonal)
    residual_elements = []
    for j in range(m):
        start = max(0, j-2)
        end = min(m, j+3)
        residual_elements.extend(residual[start:end, j])
    
    # Soma dos quadrados dos resíduos
    f = np.sum(np.array(residual_elements)**2)
    
    return f

def spmsqrt_setup(n=10):

    m = (n + 2) // 3
    
    if 3*m - 2 != n:
        raise ValueError(f"n = {n} deve satisfazer n = 3m-2")
    
    if n < 4:
        raise ValueError(f"n = {n} deve ser pelo menos 4")
    
    # Ponto inicial: 0.2*sin((1:n).^2)'
    x0 = 0.2 * np.sin(np.arange(1, n+1)**2)
    
    return x0

- Problema 16: Ults0


In [164]:
def rho(magnitude_of_velocity):
    """
    Função rho com proteção contra valores que causam raiz quadrada negativa.
    """
    # Limitar a magnitude da velocidade para evitar problemas numéricos
    magnitude_of_velocity = np.clip(magnitude_of_velocity, 0, 0.99)
    
    denominator = 1.0 - magnitude_of_velocity**2
    return 1.0 / np.sqrt(denominator)

def compute_divergence(phi_values, h):
    """
    Calcula a divergência do termo 'div[rho * nabla(phi)]'
    usando diferenças finitas.
    """
    # Assumimos que phi_values é uma grade 2D (NxN)
    
    # 1. Calcule o gradiente de phi
    grad_phi_x = np.gradient(phi_values, h, axis=0)
    grad_phi_y = np.gradient(phi_values, h, axis=1)
    
    # 2. Calcule a magnitude do gradiente
    magnitude = np.sqrt(grad_phi_x**2 + grad_phi_y**2)
    
    # 3. Calcule o rho
    density = rho(magnitude)
    
    # 4. Calcule o vetor de fluxo 'rho * nabla(phi)'
    flux_x = density * grad_phi_x
    flux_y = density * grad_phi_y
    
    # 5. Calcule a divergência do fluxo
    div_flux_x = np.gradient(flux_x, h, axis=0)
    div_flux_y = np.gradient(flux_y, h, axis=1)
    
    divergence = div_flux_x + div_flux_y
    
    return divergence

def ults0_objective(phi_1d, grid_shape, h):

    try:
        # Remodelar o array 1D para a grade 2D
        phi_values = phi_1d.reshape(grid_shape)
        
        # Calcule o lado esquerdo da equação (3.1)
        left_side = compute_divergence(phi_values, h)
        
        # Verificar se há valores NaN ou infinitos
        if np.any(np.isnan(left_side)) or np.any(np.isinf(left_side)):
            return 1e10  # Retornar um valor alto se houver problemas numéricos
        
        # Retorne a soma dos quadrados dos resíduos
        return np.sum(left_side**2)
    
    except Exception as e:
        print(f"Erro na função objetivo: {e}")
        return 1e10

# Minimizando usando o L-BFGS-B

In [165]:
from scipy.optimize import minimize

- Trigonometric:

In [166]:
n = 5
x0 = np.ones(n)
x0 = x0/n
res = minimize(trigonometric_ext_objective, x0, method='L-BFGS-B')

print("Resultado da otimização:")
print("x* =", res.x)
print("Valor da função objetivo =", res.fun)

Resultado da otimização:
x* = [0.10474191 0.11179985 0.12146511 0.35279539 0.19218876]
Valor da função objetivo = 2.6812660261121253e-08


- Extended Rosenbrock:

In [167]:
# ponto inicial
x0 = np.array([0.5, 0.5, 0.5, 0.5])  # n precisa ser par; não consegui indentificar qual é o ponto inicial no paper

res = minimize(rosenbrock_ext_objective, x0, method="L-BFGS-B")

print("x* =", res.x)
print("f(x*) =", res.fun)

x* = [1.0000112  1.00002246 0.99998294 0.9999658 ]
f(x*) = 4.1745570783568797e-10


- Extended Powell:

In [168]:
x0 = np.array([3.0, -1.0, 0.0, 1.0])

res = minimize(powell_singular_ext_objective, x0, method="L-BFGS-B")

print("x* =", res.x)
print("f(x*) =", res.fun)

x* = [ 5.20354297e-04 -5.40950257e-05  5.64219819e-03  5.64620889e-03]
f(x*) = 2.3936085771019854e-08


- QOR:

In [169]:
# Função objetivo "pronta" só em termos de x
def objective_qor(x):
    return f_qor(x, a, B, d, A_sets, B_sets)

# Dimensão do problema (50 variáveis)
n = 50

# Ponto inicial: vetor de zeros
x0 = np.zeros(n)

# Otimização
res = minimize(objective_qor, x0, method="L-BFGS-B")

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 24
x* = [ 0.59235194 -0.71119642  0.06294984 -2.65130089  1.58363972  4.02263938
  0.14360236  1.92470896 -0.06265392 -0.63941546  0.59874008  0.44592406
  1.28086251  0.71002692 -0.8775762  -3.03476637 -1.38643225  0.7405886
 -7.11236591  1.37436371  3.57044619  2.2655167   2.56624001  3.75496691
 -2.32659313  1.2112982   1.37240434  2.31100057 -0.73686734 -0.4757296
  0.69361576 -2.39885944  3.80569826  2.12269091 -3.20129753  1.10636117
  0.35492553 -1.58131176 -1.47638038  4.30993893  0.68168804  2.35414146
 -1.03420308  1.97816653 -0.33819197 -0.35798847 -3.51772534 -0.09979968
 -3.30409981  1.19738166]
Valor mínimo encontrado: 1175.4722232049098


- GOR:

In [170]:
# Função objetivo "pronta" só em termos de x
def objective_gor(x):
    return f_gor(x, a, B, d, A_sets, B_sets)

# Dimensão do problema (50 variáveis)
n = 50

# Ponto inicial: vetor de zeros
x0 = np.zeros(n)

# Otimização
res = minimize(objective_gor, x0, method="L-BFGS-B")

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 79
x* = [-1.79862431 -0.3434098  -3.04030607 -0.10580007  7.3627922   1.16102615
  3.88851806  0.25989856  0.46266803  0.2731113   0.22983314 -0.06379558
 -0.40471948 -1.74877571 -7.14853941 -0.83448447 -1.09729555 -9.5109143
 -2.07389786  9.44081847  1.36223337 -0.47580838  8.46554019 -5.54920688
  0.87607907  0.66781537  4.12257373  0.45901708 -0.17045026  0.71152347
 -1.60756313  5.19380631  8.94919989 -2.28500419  1.31748682  0.19874809
 -1.01152315 -1.59627525 10.65663252  4.081367    4.11000201 -6.3236745
  2.14936617 -0.57874344  0.47942228 -5.34299006 -2.50410865 -0.26764842
  6.36887311 -0.18490951]
Valor mínimo encontrado: 1381.1182142281775


- PSP:

In [171]:
# Função objetivo "pronta" só em termos de x
def objective_psp(x):
    return f_psp(x, a, B, d, A_sets, B_sets)

# Dimensão do problema (50 variáveis)
n = 50

# Ponto inicial: vetor de zeros
x0 = np.zeros(n)

# Otimização
res = minimize(objective_psp, x0, method="L-BFGS-B")

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 68
x* = [4.99980706 4.94318696 5.00288689 2.90310779 4.99576391 4.95737914
 4.99973757 3.49698404 5.00346472 4.70264263 4.99240915 4.79322285
 4.74454245 0.98387689 5.27087475 1.08276437 3.71376529 5.01971425
 2.73606544 2.64699594 3.40916243 4.99023264 0.97164812 4.96905179
 2.23342282 3.58367767 5.04262637 5.00281552 4.25261216 4.99460987
 5.00065149 4.93392625 1.45516803 5.02844814 4.99561912 4.5325422
 3.7064933  4.51298717 5.89486963 5.00542946 4.11397045 1.8365657
 4.99990919 3.43405045 4.99069183 1.64664123 3.44890633 3.60838614
 1.73754467 5.0081376 ]
Valor mínimo encontrado: 202.04849146512927


- TRIDIAGONAL:

In [172]:
# Função objetivo
n = 10
x0 = np.ones(n)

res = minimize(tridia_objective, x0, method="L-BFGS-B")

print("x* =", res.x)
print("f(x*) =", res.fun)


print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

x* = [1.00000105 0.49999892 0.25000078 0.12500014 0.06250054 0.03124897
 0.01562471 0.00781186 0.00390736 0.00195288]
f(x*) = 3.806211768953026e-11
Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 15
x* = [1.00000105 0.49999892 0.25000078 0.12500014 0.06250054 0.03124897
 0.01562471 0.00781186 0.00390736 0.00195288]
Valor mínimo encontrado: 3.806211768953026e-11


- ENGVAL1:

In [173]:
n = 10
x0 = 2 * np.ones(n)
res = minimize(engval1_objective, x0, method='L-BFGS-B')


print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 15
x* = [9.01035119e-01 5.45879938e-01 6.51210358e-01 6.24071756e-01
 6.31963382e-01 6.27800544e-01 6.36567313e-01 6.05287030e-01
 7.17032519e-01 3.31731053e-06]
Valor mínimo encontrado: 9.177469957554894


- Extended Freudentstein and Roth:

In [174]:
n = 10
x0 = 1 * np.ones(n)
res = minimize(freuroth_objective, x0, method='L-BFGS-B')

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 18
x* = [12.26911872 -0.83186182 -1.50692234 -1.53467108 -1.53579776 -1.5358451
 -1.53584694 -1.53585869 -1.53614981 -1.54330632]
Valor mínimo encontrado: 1014.0640725762277


- Problema 12: Matrix Square Root 1

In [175]:
n = 16 
x0 = msqrtals_setup(n)
res = minimize(msqrtals_objective, x0, method='L-BFGS-B')

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)      

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 44
x* = [ 0.84153449 -0.13241686 -0.62991331 -0.60208168 -0.7567417  -0.99178156
 -0.50636944  0.9394918   0.41221018 -0.95377925  0.99883134 -0.93008448
 -0.28775748  0.9199821  -0.49101942 -0.99929431]
Valor mínimo encontrado: 7.620865747173693e-09


- Problema 13: Matrix Square Root 2

In [176]:
n = 16  
x0 = msqrtbls_setup(n)
res = minimize(msqrtbls_objective, x0, method='L-BFGS-B')

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)      

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 60
x* = [ 0.84052162 -0.13650435 -0.00182723 -0.60347397 -0.7547155  -0.990452
 -0.50658165  0.93777665  0.41469556 -0.95404589  1.00000513 -0.92404561
 -0.28633384  0.9210025  -0.49205254 -1.00092714]
Valor mínimo encontrado: 4.143825281621554e-07


- Problema 14: LMINSURF

In [177]:
n = 16  
x0, xlower, xupper = lminsurf_setup(n)
res = minimize(lminsurf_objective, x0, method='L-BFGS-B', 
                bounds=list(zip(xlower, xupper)))

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)                

Sucesso: True
Mensagem: CONVERGENCE: NORM OF PROJECTED GRADIENT <= PGTOL
Número de iterações: 9
x* = [ 1.          3.66666667  6.33333333  9.          2.33333333  4.99994032
  7.66680433 10.33333333  3.66666667  6.33343306  9.00009033 11.66666667
  5.          7.66666667 10.33333333 13.        ]
Valor mínimo encontrado: 9.000000001442865


- Problema 15: Sparse Matrix Square Root

In [178]:
n = 10  
x0 = spmsqrt_setup(n)
res = minimize(spmsqrt_objective, x0, method='L-BFGS-B')

print("Sucesso:", res.success)
print("Mensagem:", res.message)
print("Número de iterações:", res.nit)
print("x* =", res.x)
print("Valor mínimo encontrado:", res.fun)      

Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 21
x* = [ 0.84147003 -0.75677541  0.41212499 -0.28789727 -0.13235317 -0.99178195
 -0.95375087  0.92002464 -0.62988726 -0.50636601]
Valor mínimo encontrado: 3.023049288729955e-10


- Ults0

In [179]:
N = 8  # Grade pequena para teste
h = 1.0 / (N - 1)  
grid_shape = (N, N)

# Ponto inicial para a otimização - usar valores pequenos em vez de zero
np.random.seed(42)  # Para reprodutibilidade
initial_phi_guess = 0.1 * np.random.randn(N * N)

# Adicionar limites para evitar valores muito grandes
bounds = [(-5, 5) for _ in range(N * N)]

print(f"Grade: {N}x{N}")
print(f"Ponto inicial (primeiros 5 valores): {initial_phi_guess[:5]}")

result = minimize(
    fun=ults0_objective,
    x0=initial_phi_guess,
    args=(grid_shape, h),
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 500, 'ftol': 1e-6}
)

print("Sucesso:", result.success)
print("Mensagem:", result.message)
print("Número de iterações:", result.nit)
print("x* =", result.x)
print("Valor mínimo encontrado:", result.fun)      


Grade: 8x8
Ponto inicial (primeiros 5 valores): [ 0.04967142 -0.01382643  0.06476885  0.15230299 -0.02341534]
Sucesso: True
Mensagem: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Número de iterações: 81
x* = [ 0.00117796  0.00900163 -0.00112987 -0.00401418 -0.02210035 -0.03103718
 -0.04792263 -0.05666763 -0.01718072 -0.00933459 -0.01397703 -0.01471621
 -0.02542689 -0.03079041 -0.0414984  -0.04692948 -0.01763105 -0.01382295
 -0.01734187 -0.01873677 -0.0263465  -0.0312494  -0.03937916 -0.04528951
 -0.02874507 -0.02494663 -0.02520322 -0.02500561 -0.02749243 -0.0300032
 -0.03372959 -0.03776519 -0.0222899  -0.02410908 -0.02435464 -0.02614006
 -0.02664207 -0.02996136 -0.03224442 -0.03836098 -0.03157435 -0.03342858
 -0.03028713 -0.03035652 -0.0255655  -0.02638547 -0.02409603 -0.02796605
 -0.0304283  -0.03575174 -0.03127503 -0.03175793 -0.02366426 -0.02390323
 -0.0188309  -0.02340138 -0.04416494 -0.04891586 -0.03878483 -0.03656926
 -0.02058405 -0.01720328 -0.00523372 -0.00612495]
Valor